In [1]:
# Mount Google drive to read data from it
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install gunicorn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 3.2 MB/s eta 0:00:00


In [8]:
!pip install Flask SQLAlchemy


In [29]:
import json
import csv

# Load JSON data
file_path = '/content/drive/MyDrive/THIRUKURAL/thirukural_git.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)['kurals']  # Adjust this depending on the structure of your JSON

# Prepare to write to CSV
csv_file_path = 'output.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['chapter', 'number', 'section', 'kural', 'meaning_ta_mu_va', 'meaning_ta_salamon', 'meaning_en']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for item in data:
        # Prepare data, combining kural parts
        kural_combined = ' '.join(item['kural'])

        row = {
            'chapter': item['chapter'],
            'number': item['number'],
            'section': item['section'],
            'kural': kural_combined,
            'meaning_ta_mu_va': item['meaning'].get('ta_mu_va', ''),
            'meaning_ta_salamon': item['meaning'].get('ta_salamon', ''),
            'meaning_en': item['meaning'].get('en', '')
        }
        writer.writerow(row)

print(f"Data has been written to {csv_file_path}")



Data has been written to output.csv


In [30]:
import json
import sqlite3

# Load JSON data
file_path = '/content/drive/MyDrive/THIRUKURAL/thirukural_git.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)['kurals']  # Adjust this depending on the structure of your JSON

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('kurals.db')
c = conn.cursor()

# Define the table schema
c.execute('''
CREATE TABLE IF NOT EXISTS kural (
    id INTEGER PRIMARY KEY,
    chapter TEXT,
    number INTEGER,
    section TEXT,
    kural TEXT,
    meaning_ta_mu_va TEXT,
    meaning_ta_salamon TEXT,
    meaning_en TEXT
)
''')

# Insert data into the table
for item in data:
    kural_combined = ' '.join(item['kural'])  # Combine the kural parts into a single string

    c.execute('''
    INSERT INTO kural (chapter, number, section, kural, meaning_ta_mu_va, meaning_ta_salamon, meaning_en) VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        item['chapter'],
        item['number'],
        item['section'],
        kural_combined,
        item['meaning'].get('ta_mu_va', ''),
        item['meaning'].get('ta_salamon', ''),
        item['meaning'].get('en', '')
    ))

# Commit changes and close the connection
conn.commit()
conn.close()
